In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import pandas as pd

df = pd.read_csv('/home/dariast/Disinfo_Russian_Media/data/cleaned_data.csv')
df["class"].value_counts()

class
disinformation    3757
trustworthy        336
Name: count, dtype: int64

In [6]:
df

,debunk_id,keywords,article_id,article_publisher,article_domain,article_url,article_language,debunk_date,class,text,limited_text
0,bf499726-dd35-460e-87bd-f7f158525218,Nazi/Fascist,9ea85db2-7f64-5323-ab74-bb198bcc1736,радіо свобода,radiosvoboda.org,https://www.radiosvoboda.org/a/donbass--realii...,Russian,01-06-2022,trustworthy,"Пресса о Донбассе Москве нужна не капитуляция,...",Пресса о Донбассе Москве нужна не капитуляция ...
1,c0c10537-4e54-44fe-8b8d-69ba3b3252c9,"Encircling Russia, Anti-Russian, Caucasus, Eur...",0124f417-0c49-406a-a613-8f85939c26dd,sputnik,ru-armeniasputnik-am.translate.goog,https://ru-armeniasputnik-am.translate.goog/20...,Russian,07-03-2023,disinformation,И.о постпреда РФ миссия ЕС в Армении попытка з...,И.о постпреда РФ миссия ЕС в Армении попытка з...
2,c0c10537-4e54-44fe-8b8d-69ba3b3252c9,"Encircling Russia, Anti-Russian, Caucasus, Eur...",fe896ba7-984e-4ef7-98b0-6af400dc2179,tlgrm.ru,tlgrm.ru,https://tlgrm.ru/channels/@SputnikARM,Russian,07-03-2023,disinformation,Армения Новости Посты и статистика канала Лучш...,Армения Новости Посты и статистика канала Лучш...
3,c0c10537-4e54-44fe-8b8d-69ba3b3252c9,"Encircling Russia, Anti-Russian, Caucasus, Eur...",933c67c4-169e-4044-b29a-646e9ced0508,eadaily,eadaily.com,https://eadaily.com/ru/news/2023/03/07/missiya...,Russian,07-03-2023,disinformation,Миссия раздора ЕС пытается закрепиться на пост...,Миссия раздора ЕС пытается закрепиться на пост...
4,c0c10537-4e54-44fe-8b8d-69ba3b3252c9,"Encircling Russia, Anti-Russian, Caucasus, Eur...",bdc5b6fb-c64b-40d6-aa19-f5cc9be825de,sputnik,ru.armeniasputnik.am,https://ru.armeniasputnik.am/20230307/io-postp...,Russian,07-03-2023,disinformation,И.о постпреда РФ миссия ЕС в Армении попытка з...,И.о постпреда РФ миссия ЕС в Армении попытка з...
...,...,...,...,...,...,...,...,...,...,...,...
4088,d8a3a88e-ed50-4f75-bfc5-f7655f30dc93,"Rail-roads, Corruption, NATO, Russophobia",e1acf75f-c2ca-4370-9013-bc5daa9eb3eb,sputnik,lt.sputniknews.ru,https://lt.sputniknews.ru/politics/20210128/14...,Russian,27-01-2021,disinformation,Скандал с битьем посуды Прибалтика пошла на ша...,Скандал с битьем посуды Прибалтика пошла на ша...
4089,ebc5dcde-9bc5-4dfa-9191-63070677ed30,"Global Compact for Migration, Emmanuel Macron,...",9ddf25ca-8ddd-4b74-a295-28b014ecb7d5,tsargrad,tsargrad.tv,https://tsargrad.tv/articles/makron-i-merkel-n...,Russian,04-12-2018,disinformation,Макрон и Меркель наводнят Европу миллионами ми...,Макрон и Меркель наводнят Европу миллионами ми...
4090,6baa9797-ee7b-4f83-8d34-cd78b1a7cd00,"Military, NATO",2cbd4fd7-1b82-411f-a331-cea6abcfa2f3,interfax.ru,interfax.ru,https://www.interfax.ru/world/640706?utm_sourc...,Russian,04-12-2018,disinformation,В ЛНР заявили о военных в натовской форме у ли...,В ЛНР заявили о военных в натовской форме у ли...
4091,6baa9797-ee7b-4f83-8d34-cd78b1a7cd00,"Military, NATO",a2f4039b-6b10-467f-9bd3-38c3fe20cc96,ukraina.ru,ukraina.ru,https://ukraina.ru/news/20181204/1021967878.html,Russian,04-12-2018,disinformation,Разведка ЛНР Новая бригада натовцев прибыла в ...,Разведка ЛНР Новая бригада натовцев прибыла в ...


In [2]:
api_key = os.getenv('API_KEY')
base_url = os.getenv('BASE_URL')

client = openai.OpenAI(
    base_url=base_url,
    api_key=api_key,
)

In [14]:
response = client.chat.completions.create(
    model="gemma2:9b",
    temperature=0.7,
    max_tokens=256,
    messages=[
        {"role": "system", "content": "You are a helpful assistant who is fluent in Russian."},
        {"role": "user", "content": "Напиши мне что-нибудь по-русски."},
        {"role": "assistant", "content": "Конечно! Вот пример текста на русском языке."},
        {"role": "user", "content": "Как сказать 'hello' по-русски?"},
    ],
)

print("Response:")
print(response.choices[0].message.content)

Response:
"Привет" (Privet) - это самый распространенный способ сказать "hello" по-русски. 

Можно также использовать:

* **Здравствуйте** (Zdravstvuyte) - более формальный вариант, используемый в официальных ситуациях.
* **Добрый день** (Dobryy den') - используется днем.


Какой вариант вам подходит?


In [25]:
import json
from tqdm import tqdm
path_prompts = r"..\prompts\prompts_json\zero_shot.json"

with open(path_prompts, 'r', encoding='utf-8') as file:
    prompts = json.load(file)
system_prompt = prompts[0]['content']
responses = []
for _, prompt in enumerate(tqdm(prompts[1:],  desc="Processing prompts")):
    text = prompt['content']
    print(f"Prompt: {text}")
    response = client.chat.completions.create(
        model="mistral-nemo:12b",
        temperature=0, # make the output deterministic
        max_tokens=2, # make sure the model generates the expected output
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": text}]
    )

    responses.append(response.choices[0].message.content)

for i, resp in enumerate(responses, 1):
    print(f"Response {i}: {resp}") 


Processing prompts:   0%|          | 0/50 [00:00<?, ?it/s]

Prompt: """Тебе дан текст на русском языке. Твоя задача внимательно прочитать данный ниже текст обозначенный ``` и классифицировать, является ли данный текст дезинформацией.
Дезинформация — заведомо ложная или искажённая информация.
Виды дезинформации:
- введение в заблуждение конкретного лица или группы лиц (в том числе и целой нации);
- манипулирование (поступками одного человека или группы лиц);
- создание общественного мнения относительно какой-то проблемы или объекта.

Если текст содержит дезинформацию, ответь "disinformation";
Если текст НЕ содержит дезинформацию, ответь "trustworthy".
Ответ должен содержать только одно из этих двух слов: "disinformation" или "trustworthy". 

DO NOT ADD ANY EXTRA WORDS TO YOUR ANSWER.
ANSWER ONLY IN ENGLISH. 
CHOOSE EITHER "disinformation" OR "trustworthy". 
DO NOT PROVIDE ENGLISH TRANSLATION.

```
Фильтр для наблюдателей и другие правила голосования по Конституции Русская служба , Русская служба Перейти к содержанию Разделы Главная Война в Украи

Processing prompts:   2%|▏         | 1/50 [00:08<06:44,  8.25s/it]

Prompt: """Тебе дан текст на русском языке. Твоя задача внимательно прочитать данный ниже текст обозначенный ``` и классифицировать, является ли данный текст дезинформацией.
Дезинформация — заведомо ложная или искажённая информация.
Виды дезинформации:
- введение в заблуждение конкретного лица или группы лиц (в том числе и целой нации);
- манипулирование (поступками одного человека или группы лиц);
- создание общественного мнения относительно какой-то проблемы или объекта.

Если текст содержит дезинформацию, ответь "disinformation";
Если текст НЕ содержит дезинформацию, ответь "trustworthy".
Ответ должен содержать только одно из этих двух слов: "disinformation" или "trustworthy". 

DO NOT ADD ANY EXTRA WORDS TO YOUR ANSWER.
ANSWER ONLY IN ENGLISH. 
CHOOSE EITHER "disinformation" OR "trustworthy". 
DO NOT PROVIDE ENGLISH TRANSLATION.

```
Байден хочет легализовать сбыт наркотиков на Украине АНТИФАШИСТ информационное агентство Россия и Белоруссия проведут глобальный обмен с Западом . Ког

Processing prompts:   4%|▍         | 2/50 [00:12<04:32,  5.68s/it]

...

Prompt: """Тебе дан текст на русском языке. Твоя задача внимательно прочитать данный ниже текст обозначенный ``` и классифицировать, является ли данный текст дезинформацией.
Дезинформация — заведомо ложная или искажённая информация.
Виды дезинформации:
- введение в заблуждение конкретного лица или группы лиц (в том числе и целой нации);
- манипулирование (поступками одного человека или группы лиц);
- создание общественного мнения относительно какой-то проблемы или объекта.

Если текст содержит дезинформацию, ответь "disinformation";
Если текст НЕ содержит дезинформацию, ответь "trustworthy".
Ответ должен содержать только одно из этих двух слов: "disinformation" или "trustworthy". 

DO NOT ADD ANY EXTRA WORDS TO YOUR ANSWER.
ANSWER ONLY IN ENGLISH. 
CHOOSE EITHER "disinformation" OR "trustworthy". 
DO NOT PROVIDE ENGLISH TRANSLATION.

```
Прольется кровь политолог рассказал , как Украина скоро получит второй Донбасс МК МК СпортМК АвтоВзгляд Охотники.ру . Промокоды Города для жизни Предн

Processing prompts:  98%|█████████▊| 49/50 [04:38<00:07,  7.63s/it]

Prompt: """Тебе дан текст на русском языке. Твоя задача внимательно прочитать данный ниже текст обозначенный ``` и классифицировать, является ли данный текст дезинформацией.
Дезинформация — заведомо ложная или искажённая информация.
Виды дезинформации:
- введение в заблуждение конкретного лица или группы лиц (в том числе и целой нации);
- манипулирование (поступками одного человека или группы лиц);
- создание общественного мнения относительно какой-то проблемы или объекта.

Если текст содержит дезинформацию, ответь "disinformation";
Если текст НЕ содержит дезинформацию, ответь "trustworthy".
Ответ должен содержать только одно из этих двух слов: "disinformation" или "trustworthy". 

DO NOT ADD ANY EXTRA WORDS TO YOUR ANSWER.
ANSWER ONLY IN ENGLISH. 
CHOOSE EITHER "disinformation" OR "trustworthy". 
DO NOT PROVIDE ENGLISH TRANSLATION.

```
Смолин в Европе будут упираться ногами и руками в вопросе продажи российской вакцины .. , Армения Армения рус Армения рус Брифинг Захаровой Смолин в Е

Processing prompts: 100%|██████████| 50/50 [04:49<00:00,  5.79s/it]

Response 1: Итак
Response 2: Here are
Response 3: disinformation
Response 4: Цар
Response 5: disinformation
Response 6: Тера
Response 7: disinformation
Response 8: Изви
Response 9: Я ви
Response 10: Изви
Response 11: disinformation
Response 12: disinformation
Response 13: Изви
Response 14: disinformation
Response 15: **После
Response 16: disinformation
Response 17: Изви
Response 18: disinformation
Response 19: В тек
Response 20: Феде
Response 21: Итак
Response 22: disinformation
Response 23: disinformation
Response 24: disinformation
Response 25: Изви
Response 26: Изви
Response 27: disinformation
Response 28: В ва
Response 29: Изви
Response 30: disinformation
Response 31: Изви
Response 32: disinformation
Response 33: disinformation
Response 34: Изви
Response 35: **К
Response 36: disinformation
Response 37: disinformation
Response 38: disinformation
Response 39: Вкра
Response 40: disinformation
Response 41: disinformation
Response 42: disinformation
Response 43: disinformation
Response 

In [2]:
response = client.chat.completions.create(
    model="llama3.1:8b",
    temperature=0.7,
    max_tokens=256,
    messages=[
        {"role": "system", "content": "You are a helpful assistant who is fluent in Russian."},
        {"role": "user", "content": "Напиши мне что-нибудь по-русски. Не переводи ответ на английский."},
    ],
)

print("Response:")
print(response.choices[0].message.content)

Response:
Давайте поговорим! Как вы чувствовали себя сегодня? У вас был замечательный день или, может быть, всё было не так хорошо?
